In [1]:
# %pip install autogluon

In [2]:
# READ dataset\CEAS_08.csv and make a train and test split then save to new csv files
import os
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('dataset/CEAS_08.csv')
train, test = train_test_split(df, test_size=0.2)

# make a new dir for autogloun to read the data
os.makedirs('dataset_autogluon', exist_ok=True)

train.to_csv('dataset_autogluon/train.csv', index=False)
test.to_csv('dataset_autogluon/test.csv', index=False)

C:\Users\paula\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
from autogluon.tabular import TabularDataset, TabularPredictor

data_root = 'dataset_autogluon/'
train_data = TabularDataset(data_root + 'train.csv')
test_data = TabularDataset(data_root + 'test.csv')

predictor = TabularPredictor(label='label').fit(train_data=train_data)

No path specified. Models will be saved in: "AutogluonModels\ag-20241003_000618"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.5
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          32
Memory Avail:       47.80 GB / 63.75 GB (75.0%)
Disk Space Avail:   3152.20 GB / 3725.17 GB (84.6%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial 

In [4]:
predictions = predictor.predict(test_data)
predictions.head()

0    1
1    0
2    0
3    0
4    1
Name: label, dtype: int64

In [5]:
predictor.evaluate(test_data, silent=True)

{'accuracy': 0.9948920955178138,
 'balanced_accuracy': 0.9950410126196556,
 'mcc': 0.9896183049345015,
 'roc_auc': 0.9998112745286708,
 'f1': 0.995475113122172,
 'precision': 0.997054158169046,
 'recall': 0.9939010616670432}

In [6]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost,0.995020,0.9972,accuracy,0.794080,0.150015,12.535784,0.794080,0.150015,12.535784,1,True,7
1,WeightedEnsemble_L2,0.994892,0.9980,accuracy,1.239892,0.228064,20.095648,0.002500,0.000500,0.039007,2,True,8
2,RandomForestGini,0.994254,0.9972,accuracy,0.443312,0.077550,7.520857,0.443312,0.077550,7.520857,1,True,3
3,ExtraTreesEntr,0.994126,0.9968,accuracy,0.433464,0.067508,7.378530,0.433464,0.067508,7.378530,1,True,6
4,ExtraTreesGini,0.993871,0.9972,accuracy,0.418768,0.068009,8.983627,0.418768,0.068009,8.983627,1,True,5
5,RandomForestEntr,0.993743,0.9972,accuracy,0.393245,0.066508,5.888525,0.393245,0.066508,5.888525,1,True,4
6,KNeighborsDist,0.465969,0.4712,accuracy,5.245612,0.746475,3.946615,5.245612,0.746475,3.946615,1,True,2
7,KNeighborsUnif,0.465969,0.4716,accuracy,6.086363,0.786382,4.651834,6.086363,0.786382,4.651834,1,True,1


In [15]:
best_model = predictor.model_best
print(f"Best model: {best_model}")

Best model: WeightedEnsemble_L2


In [16]:
model_info = predictor.info()
model_info['model_info'][best_model]

{'name': 'WeightedEnsemble_L2',
 'model_type': 'WeightedEnsembleModel',
 'problem_type': 'binary',
 'eval_metric': 'accuracy',
 'stopping_metric': 'accuracy',
 'fit_time': 0.03900718688964844,
 'num_classes': 2,
 'quantile_levels': None,
 'predict_time': 0.0004999637603759766,
 'val_score': 0.998,
 'hyperparameters': {'use_orig_features': False,
  'max_base_models': 25,
  'max_base_models_per_type': 5,
  'save_bag_folds': True},
 'hyperparameters_fit': {},
 'hyperparameters_nondefault': ['save_bag_folds'],
 'ag_args_fit': {'max_memory_usage_ratio': 1.0,
  'max_time_limit_ratio': 1.0,
  'max_time_limit': None,
  'min_time_limit': 0,
  'valid_raw_types': None,
  'valid_special_types': None,
  'ignored_type_group_special': None,
  'ignored_type_group_raw': None,
  'get_features_kwargs': None,
  'get_features_kwargs_extra': None,
  'predict_1_batch_size': None,
  'temperature_scalar': None,
  'drop_unique': False},
 'num_features': 2,
 'features': ['RandomForestGini', 'XGBoost'],
 'feature

In [17]:
model_info['model_info']['XGBoost']

{'name': 'XGBoost',
 'model_type': 'XGBoostModel',
 'problem_type': 'binary',
 'eval_metric': 'accuracy',
 'stopping_metric': 'accuracy',
 'fit_time': 12.535783529281616,
 'num_classes': 2,
 'quantile_levels': None,
 'predict_time': 0.15001463890075684,
 'val_score': 0.9972,
 'hyperparameters': {'n_estimators': 10000,
  'learning_rate': 0.1,
  'n_jobs': -1,
  'proc.max_category_levels': 100,
  'objective': 'binary:logistic',
  'booster': 'gbtree'},
 'hyperparameters_fit': {'n_estimators': 174},
 'hyperparameters_nondefault': [],
 'ag_args_fit': {'max_memory_usage_ratio': 1.0,
  'max_time_limit_ratio': 1.0,
  'max_time_limit': None,
  'min_time_limit': 0,
  'valid_raw_types': ['bool', 'int', 'float', 'category'],
  'valid_special_types': None,
  'ignored_type_group_special': None,
  'ignored_type_group_raw': None,
  'get_features_kwargs': None,
  'get_features_kwargs_extra': None,
  'predict_1_batch_size': None,
  'temperature_scalar': None},
 'num_features': 6998,
 'features': ['urls',

In [18]:
model_info['model_info']['RandomForestGini']

{'name': 'RandomForestGini',
 'model_type': 'RFModel',
 'problem_type': 'binary',
 'eval_metric': 'accuracy',
 'stopping_metric': 'accuracy',
 'fit_time': 7.520857095718384,
 'num_classes': 2,
 'quantile_levels': None,
 'predict_time': 0.07754969596862793,
 'val_score': 0.9972,
 'hyperparameters': {'n_estimators': 300,
  'max_leaf_nodes': 15000,
  'n_jobs': -1,
  'random_state': 0,
  'bootstrap': True,
  'criterion': 'gini'},
 'hyperparameters_fit': {'n_estimators': 300},
 'hyperparameters_nondefault': ['criterion'],
 'ag_args_fit': {'max_memory_usage_ratio': 1.0,
  'max_time_limit_ratio': 1.0,
  'max_time_limit': None,
  'min_time_limit': 0,
  'valid_raw_types': ['bool', 'int', 'float', 'category'],
  'valid_special_types': None,
  'ignored_type_group_special': None,
  'ignored_type_group_raw': None,
  'get_features_kwargs': None,
  'get_features_kwargs_extra': None,
  'predict_1_batch_size': None,
  'temperature_scalar': None},
 'num_features': 6998,
 'features': ['urls',
  'sender',